In [0]:
import pandas as pd
import numpy as np
from collections import Iterable

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cd drive

In [0]:
!ls

drive  sample_data


In [0]:
tcedf = pd.read_csv('/content/drive/My Drive/TCE-DAT_historic-exposure_1950-2015.csv',skiprows = 6)
tcedf = tcedf[tcedf['year'] >= 1980]

In [0]:
emdatdf = pd.read_csv('/content/drive/My Drive/emdat_cleaned.csv')
emdatdf = emdatdf[emdatdf['Year'] >= 1980 ]

In [27]:
tcedf.shape, emdatdf.shape

((3118, 15), (2866, 44))

In [0]:
tcedf.head(10)

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets
2217,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,451029.0,1.777832e+09,9653.0,3.880521e+07,0.0,0.0
2218,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,292780.0,1.716966e+10,164543.0,7.926763e+09,0.0,0.0
2219,1980,1980033S12161,UNNAMED,NaN,SP,multi,SLB,25.2,49.0,1150.0,2.863507e+06,0.0,0.000000e+00,0.0,0.0
2220,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,936704.0,1.130220e+10,460352.0,5.297437e+09,0.0,0.0
2221,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,59080.0,1.432924e+09,0.0,0.000000e+00,0.0,0.0
2222,1980,1980069S12161,SINA,NaN,SP,multi,NZL,44.4,86.3,847876.0,6.205658e+10,13217.0,2.215335e+09,0.0,0.0
2223,1980,1980073S09133,DORIS,NaN,SI,single,AUS,28.7,55.7,9356.0,3.546134e+08,0.0,0.000000e+00,0.0,0.0
2224,1980,1980075S11072,UNNAMED,NaN,SI,single,MDG,27.9,54.3,183425.0,5.018920e+07,0.0,0.000000e+00,0.0,0.0
2225,1980,1980076N06148,UNNAMED,NaN,WP,single,PHL,25.8,50.2,2134777.0,1.152364e+10,0.0,0.000000e+00,0.0,0.0
2226,1980,1980081S12170,UNNAMED,MR198003B012,SP,multi,TON,33.9,65.8,14970.0,5.529416e+07,0.0,0.000000e+00,0.0,0.0


In [0]:
emdatdf.head(10)

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Event name,Entry criteria,Country name,Region,Continent,EU member,Location,Origin,Associated disater,Associated disater 2,OFDA response,Appeal,Declaration,Aid contribution ('000$),Magnitude (value),Magnitude (scale),Latitude,Longitude,Local time,River basin,Start year,Start month,Start day,End year,End month,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI
447,2198,1980,78,HTI,Natural,Meteorological,Storm,Tropical cyclone,NaN,Allen,Kill,Haiti,Caribbean,Americas,No,"South-West, Port-Au-Prince",NaN,Flood,NaN,Yes,No,No,NaN,270.0,Kph,NaN,NaN,NaN,NaN,1980,8.0,5.0,1980,8.0,5.0,220.0,NaN,330000.0,835000.0,1165000.0,NaN,NaN,400000.0,32.233893
448,2201,1980,220,HKG,Natural,Meteorological,Storm,Convective storm,Lightning/Thunderstorms,NaN,Affect,Hong Kong,Eastern Asia,Asia,No,"Tsuen Wan, Kwai Chung, Sha Tin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,8.0,7.0,1980,8.0,7.0,NaN,NaN,300.0,NaN,300.0,NaN,NaN,NaN,32.233893
449,2202,1980,54,BRA,Natural,Meteorological,Storm,Convective storm,Tornado,NaN,Kill,Brazil,South America,Americas,No,Irati town,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,6.0,25.0,1980,6.0,25.0,15.0,48.0,NaN,NaN,48.0,NaN,NaN,NaN,32.233893
450,2203,1980,234,DNK,Natural,Meteorological,Storm,Convective storm,Severe storm,NaN,Declar,Denmark,Northern Europe,Europe,Yes,N-W Jutland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,12.0,24.0,1980,12.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.233893
451,2213,1980,286,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Amy,Govern,Australia,Australia and New Zealand,Oceania,No,New South Wales Coast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,1.0,NaN,1980,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6966.0,32.233893
452,2214,1980,287,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Dean,Govern,Australia,Australia and New Zealand,Oceania,No,Pilbara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,2.0,NaN,1980,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40026.0,32.233893
453,2215,1980,289,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Enid,Govern,Australia,Australia and New Zealand,Oceania,No,Pilbara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,2.0,NaN,1980,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4605.0,32.233893
454,2216,1980,290,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,Govern,Australia,Australia and New Zealand,Oceania,No,"Brisbane, Queensland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,12.0,NaN,1980,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13932.0,32.233893
455,2217,1980,38,BGD,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,Kill,Bangladesh,Southern Asia,Asia,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1980,4.0,NaN,1980,4.0,NaN,11.0,50.0,1000.0,NaN,1050.0,NaN,NaN,NaN,32.233893
456,2218,1980,306,BRB,Natural,Meteorological,Storm,Tropical cyclone,NaN,Allen,Affect,Barbados,Caribbean,Americas,No,NaN,NaN,NaN,NaN,NaN,No,No,NaN,270.0,Kph,NaN,NaN,NaN,NaN,1980,7.0,31.0,1980,8.0,11.0,NaN,7.0,NaN,5000.0,5007.0,NaN,NaN,1500.0,32.233893


In [0]:
### emdat processing

In [0]:
### emdat Try dropping rows without storm names. 
emdat_noname_df = emdatdf.dropna(subset=['Event name'])

In [0]:
### Helper function - Flatten nested list
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [0]:
emdat_cols_drop = ['Latitude','Longitude','Magnitude (scale)',
                   'EU member','OFDA response', 'Appeal',
                  'Declaration', "Aid contribution ('000$)",
                  'Associated disater', 'Associated disater 2']

emdat_noname_df = emdat_noname_df.drop(columns=emdat_cols_drop)

## Some event names have year numbers. Strip this out.
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace('\d+', '')

In [0]:
### Remove words like "Hurricane", "Cyclone" e.t.c from names column
import re

to_remove = ['Hurricane', 'Tropical', 'Cyclone','depression', 'storm', 'cylone',
             'Storm', r"\(.*\)", "'", '"', "Typhoon", 'CYCLONE', 'cyclone', 'Tropcal', 
             'TYphoon', 'Topical', 'Depression', 'Typhhon', 'Tyhoon', 'Cclone', 
             'TRopical', 'STROM', 'strom']


for each in to_remove:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace(each, '')


'''
for each in to_remove:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.replace(each, ''))
'''

"\nfor each in to_remove:\n  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.replace(each, ''))\n"

In [0]:
split_terms = ["&", " AND ", "/", " ET ", ","]

for x in split_terms:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace(x, ' & ')


## Indexes of entries with joined names
indexes = emdat_noname_df[emdat_noname_df['Event name'].str.contains("|".join(split_terms))].index


subset_emdat = emdat_noname_df.copy().loc[indexes]
subset_emdat['Event name'] = subset_emdat['Event name'].str.split('&')


cols = subset_emdat.columns.values
cols = [v for v in cols if v != 'Event name']



temp_emdat = []

for _, row in subset_emdat.iterrows():
  for d in row['Event name']:
    temp_emdat.append(flatten([list(row[cols]), d]))


ext_subset_emdat = pd.DataFrame(temp_emdat, columns=cols + ['Event name'])


In [0]:
## Entries with Multi-Event Names
emdat_noname_df.loc[emdat_noname_df[emdat_noname_df['Event name'].str.contains("|".join(split_terms))].index]

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Event name,Entry criteria,Country name,Region,Continent,Location,Origin,Magnitude (value),Local time,River basin,Start year,Start month,Start day,End year,End month,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI
716,3519,1982,147,MDG,Natural,Meteorological,Storm,Tropical cyclone,NaN,Benedic & Electre & Frida & Justine,Kill,Madagascar,Eastern Africa,Africa,"Sambava, Antalaha, Tulear, Morondova, Miandriv...",NaN,200.0,NaN,NaN,1982,1.0,28.0,1982,3.0,25.0,92.0,NaN,NaN,70000.0,70000.0,NaN,NaN,250000.00,37.745822
791,3954,1985,21,FJI,Natural,Meteorological,Storm,Tropical cyclone,NaN,Eric & Nigel & Odette,Kill,Fiji,Melanesia,Oceania,Lautoka area,NaN,140.0,NaN,NaN,1985,1.0,17.0,1985,1.0,19.0,28.0,NaN,100000.0,NaN,100000.0,NaN,68400.0,73000.00,42.074527
884,4735,1984,19,ZAF,Natural,Meteorological,Storm,Tropical cyclone,NaN,Domoina & Imboa,Kill,South Africa,Southern Africa,Africa,"Natal, Kwazulu, Transvaal",NaN,NaN,NaN,NaN,1984,1.0,28.0,1984,2.0,4.0,64.0,NaN,500000.0,NaN,500000.0,NaN,NaN,92000.00,40.633804
893,4791,1985,20,VUT,Natural,Meteorological,Storm,Tropical cyclone,NaN,Eric & Nigel,Affect,Vanuatu,Melanesia,Oceania,NaN,NaN,NaN,NaN,NaN,1985,1.0,16.0,1985,1.0,16.0,9.0,NaN,117500.0,NaN,117500.0,NaN,NaN,173000.00,42.074527
941,5202,1989,29,COM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Kalafandji & Firinga,OFDA,Comoros (the),Eastern Africa,Africa,Anjouan,NaN,NaN,NaN,NaN,1989,1.0,24.0,1989,1.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5236.00,48.489365
1034,5952,1989,128,VNM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Brian & Angela & Dan,Kill,Viet Nam,South-Eastern Asia,Asia,"Nghe Tinh, Binh Tri Thien provinces",NaN,NaN,NaN,NaN,1989,10.0,NaN,1989,10.0,NaN,104.0,762.0,4635000.0,NaN,4635762.0,NaN,NaN,NaN,48.489365
1035,5953,1989,2,WSM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Fili & Gina,OFDA,Samoa,Polynesia,Oceania,"Countrywide, especially Savaii Isl.",NaN,NaN,NaN,NaN,1989,1.0,6.0,1989,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15500.00,48.489365
1059,6195,1990,25,PHL,Natural,Meteorological,Storm,Tropical cyclone,NaN,Ofelia & Percy,Kill,Philippines (the),South-Eastern Asia,Asia,North and Eastern Luzon,NaN,150.0,NaN,NaN,1990,6.0,22.0,1990,6.0,22.0,64.0,14.0,160000.0,NaN,160014.0,NaN,NaN,NaN,51.106795
1098,6415,1991,364,WSM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Val & Wasa,Kill,Samoa,Polynesia,Oceania,Savaii isl.,NaN,167.0,NaN,NaN,1991,12.0,7.0,1991,12.0,7.0,13.0,NaN,85000.0,3000.0,88000.0,NaN,NaN,278000.00,53.271157
1191,7055,1992,470,LAO,Natural,Meteorological,Storm,Tropical cyclone,NaN,Zeke & Fred,Kill,Lao People's Democratic Republic (the),South-Eastern Asia,Asia,Whole country,NaN,NaN,NaN,NaN,1992,7.0,10.0,1992,7.0,10.0,22.0,102.0,268775.0,NaN,268877.0,NaN,NaN,3650.00,54.884642


In [0]:
emdat_noname_df.loc[indexes]

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Event name,Entry criteria,Country name,Region,Continent,Location,Origin,Magnitude (value),Local time,River basin,Start year,Start month,Start day,End year,End month,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI
716,3519,1982,147,MDG,Natural,Meteorological,Storm,Tropical cyclone,NaN,Benedic & Electre & Frida & Justine,Kill,Madagascar,Eastern Africa,Africa,"Sambava, Antalaha, Tulear, Morondova, Miandriv...",NaN,200.0,NaN,NaN,1982,1.0,28.0,1982,3.0,25.0,92.0,NaN,NaN,70000.0,70000.0,NaN,NaN,250000.00,37.745822
791,3954,1985,21,FJI,Natural,Meteorological,Storm,Tropical cyclone,NaN,Eric & Nigel & Odette,Kill,Fiji,Melanesia,Oceania,Lautoka area,NaN,140.0,NaN,NaN,1985,1.0,17.0,1985,1.0,19.0,28.0,NaN,100000.0,NaN,100000.0,NaN,68400.0,73000.00,42.074527
884,4735,1984,19,ZAF,Natural,Meteorological,Storm,Tropical cyclone,NaN,Domoina & Imboa,Kill,South Africa,Southern Africa,Africa,"Natal, Kwazulu, Transvaal",NaN,NaN,NaN,NaN,1984,1.0,28.0,1984,2.0,4.0,64.0,NaN,500000.0,NaN,500000.0,NaN,NaN,92000.00,40.633804
893,4791,1985,20,VUT,Natural,Meteorological,Storm,Tropical cyclone,NaN,Eric & Nigel,Affect,Vanuatu,Melanesia,Oceania,NaN,NaN,NaN,NaN,NaN,1985,1.0,16.0,1985,1.0,16.0,9.0,NaN,117500.0,NaN,117500.0,NaN,NaN,173000.00,42.074527
941,5202,1989,29,COM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Kalafandji & Firinga,OFDA,Comoros (the),Eastern Africa,Africa,Anjouan,NaN,NaN,NaN,NaN,1989,1.0,24.0,1989,1.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5236.00,48.489365
1034,5952,1989,128,VNM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Brian & Angela & Dan,Kill,Viet Nam,South-Eastern Asia,Asia,"Nghe Tinh, Binh Tri Thien provinces",NaN,NaN,NaN,NaN,1989,10.0,NaN,1989,10.0,NaN,104.0,762.0,4635000.0,NaN,4635762.0,NaN,NaN,NaN,48.489365
1035,5953,1989,2,WSM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Fili & Gina,OFDA,Samoa,Polynesia,Oceania,"Countrywide, especially Savaii Isl.",NaN,NaN,NaN,NaN,1989,1.0,6.0,1989,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15500.00,48.489365
1059,6195,1990,25,PHL,Natural,Meteorological,Storm,Tropical cyclone,NaN,Ofelia & Percy,Kill,Philippines (the),South-Eastern Asia,Asia,North and Eastern Luzon,NaN,150.0,NaN,NaN,1990,6.0,22.0,1990,6.0,22.0,64.0,14.0,160000.0,NaN,160014.0,NaN,NaN,NaN,51.106795
1098,6415,1991,364,WSM,Natural,Meteorological,Storm,Tropical cyclone,NaN,Val & Wasa,Kill,Samoa,Polynesia,Oceania,Savaii isl.,NaN,167.0,NaN,NaN,1991,12.0,7.0,1991,12.0,7.0,13.0,NaN,85000.0,3000.0,88000.0,NaN,NaN,278000.00,53.271157
1191,7055,1992,470,LAO,Natural,Meteorological,Storm,Tropical cyclone,NaN,Zeke & Fred,Kill,Lao People's Democratic Republic (the),South-Eastern Asia,Asia,Whole country,NaN,NaN,NaN,NaN,1992,7.0,10.0,1992,7.0,10.0,22.0,102.0,268775.0,NaN,268877.0,NaN,NaN,3650.00,54.884642


In [0]:
emdat_noname_df.loc[~emdat_noname_df.index.isin(indexes)].shape

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Event name,Entry criteria,Country name,Region,Continent,Location,Origin,Magnitude (value),Local time,River basin,Start year,Start month,Start day,End year,End month,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI
447,2198,1980,78,HTI,Natural,Meteorological,Storm,Tropical cyclone,NaN,Allen,Kill,Haiti,Caribbean,Americas,"South-West, Port-Au-Prince",NaN,270.0,NaN,NaN,1980,8.0,5.0,1980,8.0,5.0,220.0,NaN,330000.0,835000.0,1165000.0,NaN,NaN,400000.0,32.233893
451,2213,1980,286,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Amy,Govern,Australia,Australia and New Zealand,Oceania,New South Wales Coast,NaN,NaN,NaN,NaN,1980,1.0,NaN,1980,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6966.0,32.233893
452,2214,1980,287,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Dean,Govern,Australia,Australia and New Zealand,Oceania,Pilbara,NaN,NaN,NaN,NaN,1980,2.0,NaN,1980,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40026.0,32.233893
453,2215,1980,289,AUS,Natural,Meteorological,Storm,Tropical cyclone,NaN,Enid,Govern,Australia,Australia and New Zealand,Oceania,Pilbara,NaN,NaN,NaN,NaN,1980,2.0,NaN,1980,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4605.0,32.233893
456,2218,1980,306,BRB,Natural,Meteorological,Storm,Tropical cyclone,NaN,Allen,Affect,Barbados,Caribbean,Americas,NaN,NaN,270.0,NaN,NaN,1980,7.0,31.0,1980,8.0,11.0,NaN,7.0,NaN,5000.0,5007.0,NaN,NaN,1500.0,32.233893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,24139,2019,9,THA,Natural,Meteorological,Storm,Tropical cyclone,NaN,Pabuk,Affected,Thailand,South-Eastern Asia,Asia,"Nakhon Si Thammarat, Surat Thani provinces",NaN,NaN,NaN,NaN,2019,1.0,4.0,2019,1.0,4.0,7.0,NaN,720885.0,NaN,720885.0,NaN,NaN,NaN,100.000000
3456,24140,2019,387,TWN,Natural,Meteorological,Storm,Tropical cyclone,NaN,Lekima,Kill,Taiwan (Province of China),Eastern Asia,Asia,NaN,NaN,175.0,NaN,NaN,2019,8.0,10.0,2019,8.0,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000
3457,24141,2019,165,TZA,Natural,Meteorological,Storm,Tropical cyclone,NaN,Kenneth,Kill,"Tanzania, United Republic of",Eastern Africa,Africa,"Mtwara, Lindi",NaN,NaN,NaN,NaN,2019,4.0,26.0,2019,4.0,26.0,NaN,NaN,2000000.0,NaN,2000000.0,NaN,NaN,NaN,100.000000
3458,24142,2019,412,USA,Natural,Meteorological,Storm,Tropical cyclone,NaN,Dorian,Kill,United States of America (the),Northern America,Americas,"Florida, Georgia, South Carolina, North Caroli...",NaN,150.0,NaN,NaN,2019,9.0,4.0,2019,9.0,6.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000.0,100.000000


In [0]:
emdat_noname_df.shape

(1757, 34)

In [0]:
emdat_noname_df.head(1)

,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Event name,Group,ISO,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0,Year
447,32.233893,Americas,Haiti,5.0,8.0,1980,Kill,Allen,Natural,HTI,NaN,NaN,"South-West, Port-Au-Prince",270.0,330000.0,835000.0,NaN,NaN,NaN,Caribbean,NaN,78,5.0,8.0,1980,Meteorological,NaN,Tropical cyclone,1165000.0,400000.0,220.0,Storm,2198,1980


In [0]:
ext_subset_emdat.head(1)

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Entry criteria,Country name,Region,Continent,Location,Origin,Magnitude (value),Local time,River basin,Start year,Start month,Start day,End year,End month,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI,Event name
0,3519,1982,147,MDG,Natural,Meteorological,Storm,Tropical cyclone,NaN,Kill,Madagascar,Eastern Africa,Africa,"Sambava, Antalaha, Tulear, Morondova, Miandriv...",NaN,200.0,NaN,NaN,1982,1.0,28.0,1982,3.0,25.0,92.0,NaN,NaN,70000.0,70000.0,NaN,NaN,250000.0,37.745822,Benedic


In [0]:
### Order of columns not the same. Enforce uniformity
emdat_noname_df = pd.concat([emdat_noname_df.loc[~emdat_noname_df.index.isin(indexes)], 
                             ext_subset_emdat[emdat_noname_df.columns.values]], sort=True)

In [0]:
emdat_noname_df['Event name'].unique()

array(['Allen', 'Amy', 'Dean', 'Enid', 'Joe', 'Wally', 'Daman', 'Arthur',
       'Alberto', 'Alleta', 'Orchid', 'Kim', 'Betty ', '', 'Hyacinthe',
       'Joe ', 'Ruth ', 'Phyllis', 'Thad', 'Agnes', 'St. Lidia', 'Cliff',
       'Freda', 'Gyan', 'Clara', 'Irma ', 'Lee ', 'Kelly', 'Damia',
       'Mamie', 'Nelson ', 'Bernie', 'Isaac', 'Dominic', 'Klaus',
       'Winifred', 'Beryl', 'Bess', 'Ellis', 'Judy', 'Cecil', 'Paul',
       'Frida', 'Faye', 'Nancy ', 'Ellen', 'Elinah', 'Oscar', 'Sarah',
       'Forest', 'Tico', 'Andy', 'Vera ', 'Ellen ', 'Orchid ', 'Ike',
       'Fran', 'Hortense', 'Kamisy', 'Domoina', 'Haja', 'June ', 'Agnes ',
       'Ike  and Maring', 'Warren ', ' Kate', 'Gavin', 'Tess', 'Kate',
       ' Irma', 'Pat', 'Brenda', 'Ditra', 'Dot ', 'Hal ', 'Tess ',
       'Peggy', 'Vera', 'Rajah', 'Wayne', 'Charlie', 'Danielle',
       'Honorinina', 'Peggy ', 'Wayne ', 'Dom ', 'Georgia ', 'Narge ',
       'Emily', 'Gerald', 'Ruth', 'Sally', 'Jan', 'Nina', 'Lynn',
       'Thelma', 'Ve

In [0]:
## Remove rows without event name
emdat_noname_df[emdat_noname_df['Event name'] != ""]

## Strip leading spaces .str.strip()
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.strip())

In [0]:
### It appears some observations have name as simply "Hurricane" 
## and removing these leaves the name empty. This should be dropped.
mask = emdatdf['Event name'].str.contains(r'\b(?:{})\b'.format('|'.join(to_remove)))
mask = mask.replace(np.nan, False)


## Convert names to uppercase
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.upper())

In [0]:
emdat_noname_df['Event name'].value_counts()

            30
IRMA        18
NATE        17
EMILY       15
MATTHEW     15
            ..
DANTE        1
MEKKHALA     1
BENI         1
STEVE        1
LORENZO      1
Name: Event name, Length: 736, dtype: int64

In [0]:
emdat_noname_df.shape

(1933, 34)

In [0]:
#### Ibtracs

In [0]:
ibtracdf = pd.read_csv('/content/drive/My Drive/IBTrACS-ALL-list-v04r00-lines-dbf.csv')
ibtracdf = ibtracdf[ibtracdf['year'] >= 1980]
tup_zipped = list(set(tuple(zip(ibtracdf.SID, ibtracdf.NAME))))

## Create a dictionary with ibtracs SID & storm name
SID_Namedict = {x:y for x, y in tup_zipped }

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,21,22,61,66,128,143,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
###Tce dat processing

In [0]:
tcedf_copy = tcedf.copy()

## Create column "name" using ibtracID
tcedf_copy['name'] = tcedf_copy['IBTrACS_ID'].apply(lambda x: SID_Namedict.get(str(x)))

## Drop rows without event name
tcedf_copy = tcedf_copy[tcedf_copy['name'] != "NOT_NAMED"]

# Drop rows with no names on both name & tc_name
tcedf_copy = tcedf_copy[(tcedf_copy['TC_name'] != "UNNAMED") & (tcedf_copy['name'].notna())]

In [0]:
tcedf_copy.head()

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,name
2217,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,451029.0,1.777832e+09,9653.0,3.880521e+07,0.0,0.0,PENI
2218,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,292780.0,1.716966e+10,164543.0,7.926763e+09,0.0,0.0,PAUL
2220,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,936704.0,1.130220e+10,460352.0,5.297437e+09,0.0,0.0,LAURE
2221,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,59080.0,1.432924e+09,0.0,0.000000e+00,0.0,0.0,SINA
2222,1980,1980069S12161,SINA,NaN,SP,multi,NZL,44.4,86.3,847876.0,6.205658e+10,13217.0,2.215335e+09,0.0,0.0,SINA


In [0]:
## SINA appears twice. 1990 & 1980 We would
# need to merge on both name and year. 
tcedf_copy[tcedf_copy['TC_name'] == "SINA"]

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,name
2221,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,59080.0,1.432924e+09,0.0,0.000000e+00,0.0,0.0,SINA
2222,1980,1980069S12161,SINA,NaN,SP,multi,NZL,44.4,86.3,847876.0,6.205658e+10,13217.0,2.215335e+09,0.0,0.0,SINA
3127,1990,1990327S07175,SINA,MR199011B034,SP,single,FJI,70.8,137.6,597688.0,2.549236e+09,512366.0,2.171554e+09,225854.0,920679559.0,SINA


In [0]:
## TC_NAME
tcedf_copy["TC_name"][tcedf_copy["TC_name"] == 'BELLY'] = "BETTY"
tcedf_copy["TC_name"][tcedf_copy["TC_name"] == 'BRENDA'] = "BRENDAN"




id_split = ['1980073S09133', '1987035S12160', '1988285N09318', '1989209N22130', 
            '1996282N11162', '1998036S13135', '2013130N04093']


for eachsplit in id_split:
  tcedf_copy['TC_name'][tcedf_copy['IBTrACS_ID'] == eachsplit] = tcedf_copy['name'].str.split(":").str[0]
  tcedf_copy['name'][tcedf_copy['IBTrACS_ID'] == eachsplit] = tcedf_copy['name'].str.split(":").str[1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [0]:
## Columns to keep
tce_dat_columns = ['year', 'IBTrACS_ID', 'name', 'TC_name', 'countries_affected', 'ISO3', 'v_land_SI',
                   'v_land_kn', '34kn_pop', '34kn_assets', '64kn_pop', '64kn_assets', 
                   '96kn_pop', '96kn_assets']

tcedf_copy = tcedf_copy[tce_dat_columns]                  

In [0]:
## Entries where names don't match. This is 
# either due to typos or multiple naming

tcedf_copy[["IBTrACS_ID", "TC_name", "name"]][tcedf_copy['TC_name'] != tcedf_copy['name']]

,IBTrACS_ID,TC_name,name
2223,1980073S09133,DORIS-GLORIA,GLORIA
2778,1987035S12160,UMA,VELI
2789,1987140S08170,BLANCH,BLANCH(E)
2901,1988285N09318,JOAN,MIRIAM
2902,1988285N09318,JOAN,MIRIAM
2903,1988285N09318,JOAN,MIRIAM
2904,1988285N09318,JOAN,MIRIAM
2905,1988285N09318,JOAN,MIRIAM
2906,1988285N09318,JOAN,MIRIAM
2907,1988285N09318,JOAN,MIRIAM


In [0]:
### Compare emdat & Tce dat

In [0]:
emdat_list = list(emdat_noname_df['Event name'])

tce_lists = [list(tcedf_copy['TC_name']), list(tcedf_copy['name'])]

for l in tce_lists:
  results = {}
  for i in emdat_list:
    results[i] = l.count(i) 

  count = 0
  for each in results:
    if results[each] != 0:
      count += 1

  print(count) ##478 & 476

495
493


In [0]:
emdatzip = { x:y for x, y in zip(emdat_noname_df['Event name'], emdat_noname_df['Year'])}
tcezip = { x:y for x, y in zip(tcedf_copy['TC_name'], tcedf_copy['year'])}

In [30]:
#### Rolf Merge Code

merged_df = emdat_noname_df.copy()

# merged_df = merged_df.assign()
columns_to_add = tcedf_copy.columns[3:]
for col in columns_to_add:
    merged_df[col] = None


for i, col in emdat_noname_df[emdat_noname_df['Year'] >= 1960].iterrows():
    year = col['Year']
    country_code = col['ISO']
    cyclone_name = col['Event name']
    tce = tcedf_copy.loc[tcedf_copy['ISO3'] == country_code]
    print(cyclone_name)
    if cyclone_name is None or cyclone_name != cyclone_name:
        print('no data')
        continue
    cyclone_name= cyclone_name.upper()
    if len(tce) < 1:
        print('no data')
        continue

    tce_by_year = tce.loc[tce['year'] == year]
    if len(tce_by_year) < 1:
        print('no data')
        continue

    print(i)
    tce_by_cyclone = tce.loc[tce['TC_name']== cyclone_name]
    if len(tce_by_cyclone) < 1:
        print('no data')
        continue

    new_row = col.append(tce_by_cyclone[columns_to_add].iloc[0])

    merged_df.loc[i] = new_row

merged_df.to_csv('merged.csv')

ALLEN
447
AMY
451
no data
DEAN
452
no data
ENID
453
no data
ALLEN
456
JOE
457
no data
ALLEN
458
ALLEN
459
ALLEN
460
WALLY
461
no data
ALLEN
462
JOE
463
no data
DAMAN
no data
ARTHUR
475
ALBERTO
476
ALLETA
no data
ALLEN
628
ALLEN
634
ORCHID
635
ORCHID
no data
ALLEN
638
ALLEN
639
JOE
640
no data
KIM
641
no data
BETTY
642

643
no data

644
no data

645
no data
HYACINTHE
no data
JOE
650
no data
RUTH
651
PHYLLIS
658
no data
THAD
659
AGNES
660
no data
ST. LIDIA
662
no data
CLIFF
663
FREDA
664
GYAN
665
no data
CLARA
667
IRMA
668
LEE
669
no data

670
no data

671
no data

672
no data
KELLY
no data
CLIFF
674
GYAN
675
no data
DAMIA
681
no data
ALLETA
no data
MAMIE
683
NELSON
684
BERNIE
685
ISAAC
no data
DOMINIC
687
KLAUS
695
WINIFRED
704
BERYL
707
BESS
711
ELLIS
712
JUDY
713
CECIL
no data
ELLIS
no data
PAUL
717
FRIDA
718
no data
FAYE
719

720
no data
NANCY
721

722
no data

723
no data

724
no data

725
no data
ELLEN
736
ELINAH
737
OSCAR
738
SARAH
739
no data
ELLEN
740
JOE
741
no data
FOREST
745


In [31]:
merged_df.shape

(1819, 45)

In [32]:
merged_df.shape

(1819, 45)

In [33]:
merged_df[(merged_df['Total affected'].isna() & merged_df['34kn_pop'].isna())].shape

(294, 45)

In [34]:
merged_df[(merged_df['Total affected'].isna() & merged_df['34kn_pop'].isna())].shape

(294, 45)

In [35]:
### Print number of na values for affected columns

affected_list = ['Num affected',  'Num homeless', 'Num injured']

for l in affected_list:
  print(l, " ", merged_df[merged_df[l].isna()].shape)

print(merged_df[(merged_df[affected_list[0]].isna() & merged_df[affected_list[1]].isna() & merged_df[affected_list[2]].isna())].shape)


Num affected   (701, 45)
Num homeless   (1458, 45)
Num injured   (1221, 45)
(473, 45)


In [0]:
tcedf_copy.shape, emdat_noname_df.shape

((2501, 14), (1819, 34))

In [0]:
#####################
###################
##################
### New Merge Attempt
tcedf_copy.head(2)

,year,IBTrACS_ID,name,TC_name,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets
2217,1980,1980001S13173,PENI,PENI,single,FJI,43.5,84.5,451029.0,1.777832e+09,9653.0,3.880521e+07,0.0,0.0
2218,1980,1980003S15137,PAUL,PAUL,single,AUS,51.8,100.7,292780.0,1.716966e+10,164543.0,7.926763e+09,0.0,0.0


In [0]:
emdat_noname_df.head(2)

,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Event name,Group,ISO,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0,Year
447,32.233893,Americas,Haiti,5.0,8.0,1980,Kill,ALLEN,Natural,HTI,NaN,NaN,"South-West, Port-Au-Prince",270.0,330000.0,835000.0,NaN,NaN,NaN,Caribbean,NaN,78,5.0,8.0,1980,Meteorological,NaN,Tropical cyclone,1165000.0,400000.0,220.0,Storm,2198,1980
451,32.233893,Oceania,Australia,NaN,1.0,1980,Govern,AMY,Natural,AUS,NaN,NaN,New South Wales Coast,NaN,NaN,NaN,NaN,NaN,NaN,Australia and New Zealand,NaN,286,NaN,1.0,1980,Meteorological,NaN,Tropical cyclone,NaN,6966.0,NaN,Storm,2213,1980


In [0]:
##Inner Join

merged_one = pd.merge(tcedf_copy, emdat_noname_df, left_on=['TC_name', 'year', 'ISO3'], right_on=['Event name', 'Year', 'ISO'])
merged_two = pd.merge(tcedf_copy, emdat_noname_df, left_on=['name', 'year', 'ISO3'], right_on=['Event name', 'Year', 'ISO'])
merged = pd.concat([merged_one, merged_two], sort=True).drop_duplicates()

In [0]:
merged_one.shape, merged_two.shape, merged.shape

((876, 48), (870, 48), (853, 48))

In [0]:
merged[merged['Total affected'].isna()].shape

(174, 48)

In [0]:
merged.to_csv("emdat_inner.csv")

In [0]:
merged[merged['Total affected'].isna()]

,34kn_assets,34kn_pop,64kn_assets,64kn_pop,96kn_assets,96kn_pop,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Event name,Group,IBTrACS_ID,ISO,ISO3,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,TC_name,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0,Year,countries_affected,name,v_land_SI,v_land_kn,year
1,1.365669e+10,2417472.0,4.917228e+08,79959.0,1397305.0,34.0,32.233893,Americas,Cuba,4.0,8.0,1980,Declar,ALLEN,Natural,1980214N11330,CUB,CUB,NaN,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,229,4.0,8.0,1980,Meteorological,NaN,Tropical cyclone,ALLEN,NaN,NaN,3.0,Storm,2220,1980,multi,ALLEN,63.0,122.5,1980
2,5.925760e+08,72282.0,0.000000e+00,0.0,0.0,0.0,32.233893,Americas,Dominica,4.0,8.0,1980,Declar,ALLEN,Natural,1980214N11330,DMA,DMA,NaN,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,230,4.0,8.0,1980,Meteorological,NaN,Tropical cyclone,ALLEN,NaN,NaN,NaN,Storm,2221,1980,multi,ALLEN,24.6,47.9,1980
3,2.706381e+09,273762.0,0.000000e+00,0.0,0.0,0.0,32.233893,Americas,Dominican Republic (the),4.0,8.0,1980,SigDam,ALLEN,Natural,1980214N11330,DOM,DOM,NaN,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,231,4.0,8.0,1980,Meteorological,NaN,Tropical cyclone,ALLEN,NaN,47000.0,7.0,Storm,2222,1980,multi,ALLEN,31.7,61.6,1980
4,1.909564e+08,56163.0,0.000000e+00,0.0,0.0,0.0,32.233893,Americas,Grenada,4.0,8.0,1980,SigDam,ALLEN,Natural,1980214N11330,GRD,GRD,NaN,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,305,4.0,8.0,1980,Meteorological,NaN,Tropical cyclone,ALLEN,NaN,5300.0,NaN,Storm,2224,1980,multi,ALLEN,22.6,44.0,1980
8,0.000000e+00,326247.0,0.000000e+00,182816.0,0.0,0.0,32.233893,Americas,Martinique,5.0,8.0,1980,SigDam,ALLEN,Natural,1980214N11330,MTQ,MTQ,NaN,NaN,NaN,270.0,NaN,NaN,NaN,NaN,NaN,Caribbean,NaN,307,5.0,8.0,1980,Meteorological,NaN,Tropical cyclone,ALLEN,NaN,68000.0,NaN,Storm,3121,1980,multi,ALLEN,43.3,84.1,1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,4.159030e+09,460550.0,1.743116e+08,10781.0,0.0,0.0,92.708822,Africa,Cabo Verde,31.0,8.0,2015,Waiting,FRED,Natural,2015242N12343,CPV,CPV,NaN,NaN,"Boa Vista, Brava, Cima, Fogo, Ilheu Branco, Il...",140.0,NaN,NaN,NaN,NaN,NaN,Western Africa,NaN,473,31.0,8.0,2015,Meteorological,NaN,Tropical cyclone,FRED,NaN,1100.0,9.0,Storm,21144,2015,single,FRED,36.6,71.1,2015
862,1.926384e+10,3556488.0,0.000000e+00,0.0,0.0,0.0,92.708822,Asia,Viet Nam,14.0,9.0,2015,Affected,VAMCO,Natural,2015256N16113,VNM,VNM,NaN,NaN,"Da Nang, Quang Nam (Duy Xuyen, Nong Son distri...",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,620,14.0,9.0,2015,Meteorological,NaN,Tropical cyclone,VAMCO,NaN,12800.0,11.0,Storm,23911,2015,multi,VAMCO,23.2,45.1,2015
863,6.974537e+11,32499157.0,1.886408e+11,8732252.0,0.0,0.0,92.708822,Asia,China,28.0,9.0,2015,Affected,DUJUAN,Natural,2015263N14148,CHN,CHN,79000.0,NaN,"Fujian Sheng, Zhejiang Sheng provinces",NaN,NaN,NaN,NaN,NaN,NaN,Eastern Asia,NaN,458,28.0,9.0,2015,Meteorological,NaN,Tropical cyclone,DUJUAN,NaN,661000.0,NaN,Storm,21142,2015,multi,DUJUAN,48.0,93.4,2015
864,1.038558e+10,84549.0,3.644826e+09,25159.0,0.0,0.0,92.708822,Asia,Japan,28.0,9.0,2015,Affected,DUJUAN,Natural,2015263N14148,JPN,JPN,NaN,NaN,Okinawa province,NaN,NaN,NaN,NaN,NaN,NaN,Eastern Asia,NaN,458,28.0,9.0,2015,Meteorological,NaN,Tropical cyclone,DUJUAN,NaN,NaN,NaN,Storm,21723,2015,multi,DUJUAN,61.4,119.4,2015


In [0]:
merged[merged['64kn_pop'].isna()].shape

(0, 48)

In [0]:
merged[(merged['Total affected'].isna() & merged['34kn_pop'].isna())].shape

(0, 48)

In [28]:
### Print number of na values for affected columns


for l in affected_list:
  print(l, " ", merged[merged[l].isna()].shape)

print(merged[(merged[affected_list[0]].isna() & merged[affected_list[1]].isna() & merged[affected_list[2]].isna())].shape)


Num affected   (291, 48)
Num homeless   (643, 48)
Num injured   (518, 48)
(174, 48)


In [0]:
### Outer Join

##Inner Join

merged_one_outer = pd.merge(tcedf_copy, emdat_noname_df, 
                      left_on=['TC_name', 'year', 'ISO3'], 
                      right_on=['Event name', 'Year', 'ISO'], 
                      how='outer')
merged_two_outer = pd.merge(tcedf_copy, emdat_noname_df, 
                      left_on=['name', 'year', 'ISO3'], 
                      right_on=['Event name', 'Year', 'ISO'], 
                      how='outer')
merged_outer = pd.concat([merged_one_outer, merged_two_outer], sort=True).drop_duplicates()

In [0]:
merged_outer.shape

(3493, 48)

In [0]:
merged_outer.head(2)

,34kn_assets,34kn_pop,64kn_assets,64kn_pop,96kn_assets,96kn_pop,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Event name,Group,IBTrACS_ID,ISO,ISO3,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,TC_name,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0,Year,countries_affected,name,v_land_SI,v_land_kn,year
0,1.777832e+09,451029.0,3.880521e+07,9653.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980001S13173,NaN,FJI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PENI,NaN,NaN,NaN,NaN,NaN,NaN,single,PENI,43.5,84.5,1980.0
1,1.716966e+10,292780.0,7.926763e+09,164543.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980003S15137,NaN,AUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAUL,NaN,NaN,NaN,NaN,NaN,NaN,single,PAUL,51.8,100.7,1980.0


In [0]:
merged_outer[merged_outer['Total affected'].isna()].shape

(2135, 48)

In [0]:
merged_outer[(merged_outer['Total affected'].isna() & merged_outer['34kn_pop'].isna())].shape

(300, 48)

In [0]:
merged_outer.to_csv("emdat_outer.csv")

In [29]:


for l in affected_list:
  print(l, " ", merged_outer[merged_outer[l].isna()].shape)

print(merged_outer[(merged_outer[affected_list[0]].isna() & 
                    merged_outer[affected_list[1]].isna() & 
                    merged_outer[affected_list[2]].isna())].shape)

Num affected   (2365, 48)
Num homeless   (3129, 48)
Num injured   (2886, 48)
(2135, 48)
